In [1]:
%load_ext autoreload
%autoreload 2

In [121]:
from lib import *
from fun import *

# Variables

In [34]:
cardinal_grouped_polygons = 100
root = "data/elevation_files/"
datafilename = "data/polygons_and_points.csv"

# Import data

In [35]:
df = pd.read_csv(datafilename)

# Build Marks Array

In [5]:
marks = np.linspace(0,len(df),int(len(df)/cardinal_grouped_polygons)).astype(int)
a = marks[:-1]
b = marks[1:]
marks = np.vstack((a,b))
marks = marks.transpose()

In [6]:
folderfiles = listdir(root)
filenames_list = [filename for filename in folderfiles if filename.endswith("csv") ]
backfiles = filenames_list

# Retrive elevations

In [8]:
errors_counter = 0
errors_list = []
points_counter = 0

for idxmarker,bornes in enumerate(marks[:]):
    start,end = bornes
    tempdf = df[start:end]
    some_points = tempdf[["long","lat"]].to_dict(orient = "split")["data"]
    exactfilename = "file_" + str(idxmarker).zfill(5) + "_from_" + str(start) + "_to_" + str(end) + ".csv"
    tempfilename = root + exactfilename
    
    try:
        tempdata = py3dep.elevation_bycoords(some_points,crs="epsg:4326")
        exportdf = tempdf.polygonid.to_frame().reset_index(drop=True)
        exportdf["elevation"] = tempdata        
        need_to_be_zero = exportdf.elevation.isna().sum()
        if need_to_be_zero == 0:
            exportdf.to_csv(tempfilename,index = False)
        else:
            errors_counter = errors_counter + 1
            errors_list.append((idxmarker,bornes,tempdf))
    except:
        errors_counter = errors_counter + 1
        errors_list.append((idxmarker,bornes,tempdf))

    points_counter = points_counter + len(exportdf)
    progression = round(100*(points_counter / len(df))  ,  2)
    progression = str(progression) + "%"      
    clear_output(wait=True)
    print("Elevation points computed so far : ",points_counter,"(",progression,") | errors : ",errors_counter)

Elevation points computed so far :  107607 ( 100.0% ) | errors :  0


In [9]:
errors_list

[]

In [130]:
pickle.dump( errors_list, open( root + "errors_list.p", "wb" ) )

# Compute average elevation by block group and save it to a json file

In [69]:
# A lancer uniquement si l'on n'a pas relancer la construction de l'output avec randompointswithinpooly notebook
allids = np.unique(df.polygonid)
setids = np.unique(elevation_df.polygonid)
remainedids = allids[~np.isin(allids,setids)]
tempdf = df[df.polygonid.isin(remainedids)]

some_points = tempdf[["long","lat"]].to_dict(orient = "split")["data"]
tempdata = py3dep.elevation_bycoords(some_points,crs="epsg:4326")

toadddf = tempdf.polygonid.to_frame().reset_index(drop=True)
toadddf["elevation"] = tempdata

folderfiles = listdir(root)
filenames_list = [filename for filename in folderfiles if filename.endswith("csv") ]
list_of_dataframes = [pd.read_csv(root + tempfilename) for tempfilename in filenames_list]
list_of_dataframes.append(toadddf)

elevation_df = pd.concat(list_of_dataframes)
elevation_dict = elevation_df.groupby("polygonid")["elevation"].mean().to_dict()

In [73]:

with open('data/polygons_elevation.json', 'w') as f:
    json.dump(elevation_dict, f)

# Afficher la map

In [111]:
shapefilename = "data/polygons/TG00CAGRP.shp"
elevation_json_name = 'data/polygons_elevation.json'

In [112]:
# Filter part, pour pas avoir 1000 ans avant que ca s'affiche
# Point à x,y + rayon de 100 km
#centroid_mask = Point(( -118.2987,33.7866)) #LA
centroid_mask = Point(( -122.43695509472653,37.75426308044698)) #SF
box_mask = centroid_mask.buffer(distance=0.0001)
box_mask = Buffer_A_Shape(box_mask,10000)

In [113]:
with open(elevation_json_name) as json_file:
    elevation_data = json.load(json_file)
    
blocks = fiona.open(shapefilename)
blocks_polygons = [shape(item["geometry"]) for item in blocks]
blocks_filtered_array = [True if item.within(box_mask) else False for item in blocks_polygons ]

In [114]:
elevation_data = [(int(k),v) for k,v in elevation_data.items()]
polygondf = pd.DataFrame(elevation_data,columns=["polygonid","elevation"])


filterdict = {"filtered" : blocks_filtered_array,
              "polygonid" : list(range(len(blocks_filtered_array)))}
filterdf = pd.DataFrame(filterdict)
polygondf = polygondf.merge(filterdf,how="left")

In [115]:
# ATTENTION PAS D'ELEVATION POUR CERTAINS POLYGONES, A INVESTIGUER
polygondf = polygondf[(~polygondf.elevation.isna()) & (polygondf.filtered)]

In [116]:
minval = min(polygondf.elevation)
maxval = max(polygondf.elevation)
polygondf["elevation_color"] = ColorMapper(minval,maxval,polygondf.elevation,"RdYlGn_r")

In [117]:
# creation du geojson
features = []

for rowid,others in polygondf.iterrows():
    temp_elevation = others["elevation"]
    temp_color = others["elevation_color"]
    temp_id = others["polygonid"]
    
    data = {}
    data["rowid"] = int(temp_id)
    data["fillcolor"] = str(temp_color)
    data["fill_opacity"] = 0.5
    data["color"] = "white"
    data["opacity"] = 0.5
    data["elevation"] = int(temp_elevation)
    
    feature = {"type":"Feature",
               "properties":data,
               "geometry":shapely.geometry.mapping(blocks_polygons[int(temp_id)])}
    
    features.append(feature)
    
final_geojson = {"type": "FeatureCollection","features" : features}

In [118]:
centroids_array = [(pp.centroid.xy[-1][-1],pp.centroid.xy[0][-1]) for pp in blocks_polygons]
barycentre = np.mean(centroids_array,axis=0)
m = folium.Map(location = barycentre, zoom_start= 6,width="66%",tiles="Cartodbdark_matter")

stylefun = lambda x: {'fillColor' : x['properties']['fillcolor'],
                      'fillOpacity' : x['properties']['fill_opacity'],
                     'color' : x['properties']['color'],
                     'opacity' : x['properties']['opacity']}

folium.GeoJson(data=final_geojson,
               style_function= stylefun,
               smooth_factor=1,
               tooltip=folium.features.GeoJsonTooltip(fields=["rowid","elevation"],
                                                      aliases=["Polygone ID : ","Elevation (meters) : "])).add_to(m)

plugins.Fullscreen().add_to(m)
plugins.MeasureControl().add_to(m)
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('Stamen Terrain').add_to(m)
folium.LayerControl().add_to(m)